In [1]:
%load_ext dotenv
%dotenv

In [2]:
import logging
import sys
import os

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)

## 1. Preparation
### 1.1 Prepare LLM

In [3]:
from llama_index.llms import Ollama

OLLAMA_HOST = 'localhost'
OLLAMA_MODEL = 'mistral'
llm = Ollama(model=OLLAMA_MODEL, base_url="http://"+OLLAMA_HOST+":11434")

In [4]:
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(
    llm=llm, 
    # To save costs, we use a local model.
    # This will use a well-performing and fast default from Hugging Face.
    # this model has dim of 384 https://huggingface.co/BAAI/bge-small-en
    embed_model="local:BAAI/bge-small-en",
)

/Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1.2 Prepare Graph Store
`Neo4j` is supported as a graph store integration. You can persist, visualize, and query graphs using LlamaIndex and Neo4j. Furthermore, existing Neo4j graphs are directly supported using `text2cypher` and the `KnowledgeGraphQueryEngine`.

If you’ve never used Neo4j before, you can download the desktop client [here](https://neo4j.com/download/).

Once you open the client, create a new project and install the `apoc` integration. Full instructions here. Just click on your project, select `Plugins` on the left side menu, install APOC and restart your server.

In [5]:
username = "neo4j"
password = os.environ["NEO4J_PASSWORD"]
# Neo4j cloud has a generous free tier, so I use that instead of localhost
url = "bolt://localhost:7687"
# url = "bolt+s://3b2530f1.databases.neo4j.io:7687"
database = "neo4j"

In [6]:
from llama_index.graph_stores import Neo4jGraphStore

graph_store = Neo4jGraphStore(
    username=username,
    password=password,
    url=url,
    database=database,
)

In [7]:
from llama_index.storage.storage_context import StorageContext

storage_context = StorageContext.from_defaults(graph_store=graph_store)

## 2. Build the Knowledge Graph
### 2.1 Preprocess Data

In [8]:
from llama_index import download_loader

WikipediaReader = download_loader("WikipediaReader")

loader = WikipediaReader()

documents = loader.load_data(pages=['Guardians of the Galaxy Vol. 3'], auto_suggest=False)

### 2.2 Extract Triplets and Save to Graph
reference:
- [KnowledgeGraphIndex](https://docs.llamaindex.ai/en/stable/api_reference/indices/kg.html#llama_index.indices.knowledge_graph.KnowledgeGraphIndex)

In [26]:
from llama_index import KnowledgeGraphIndex

kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    service_context=service_context,
    max_triplets_per_chunk=10,
    include_embeddings=True,
    show_progress=True
    # max_object_length: int = 128,
    
    # to extract triplets, kg_triplet_extract_fn is used if not None,
    # kg_triplet_extract_fn: Optional[Callable] = None, 
    # else, the LLM from the service context is used with the kg_triple_extract_template if not None else the default triplet extract prompt
    # kg_triple_extract_template: Optional[BasePromptTemplate] = None,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Processing nodes:   0%|                                                                                                                                               | 0/16 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"



Processing nodes:   6%|████████▍                                                                                                                              | 1/16 [00:22<05:35, 22.35s/it]

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"



Processing nodes:  12%|████████████████▉                                                                                                                      | 2/16 [00:35<03:58, 17.07s/it]

...


Processing nodes:  88%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                | 14/16 [03:17<00:29, 14.77s/it]

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"



Processing nodes:  94%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋        | 15/16 [03:31<00:14, 14.55s/it]

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"



Processing nodes: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [03:42<00:00, 13.91s/it]


### Conclusions from building the KG

- we can visualize the graph in Neo4j Bloom directly on top of Neo4j local or Neo4j AuraDB
- we have 1 Node type: Entity
    ```cypher
    MATCH (n)
    RETURN distinct labels(n)[0] as label, count(n) as node_count
    ```
- 80 Relationship type
    ```cypher
    MATCH p=()-->() RETURN count(p)
    ```
- Entities only have 1 field called `id`, we don't have entity type like "Person" etc... NER would be needed


## 3. Create VectorStoreIndex for RAG

References:
- [Neo4jVectorDemo](https://docs.llamaindex.ai/en/latest/examples/vector_stores/Neo4jVectorDemo.html#)
- [Neo4jVectorStore](https://docs.llamaindex.ai/en/stable/api/llama_index.vector_stores.Neo4jVectorStore.html#llama_index.vector_stores.Neo4jVectorStore)

In [19]:
from llama_index.vector_stores import Neo4jVectorStore

# "local:BAAI/bge-small-en" has dim of 384 https://huggingface.co/BAAI/bge-small-en
embed_dim = len(service_context.embed_model.get_text_embedding("I want to check embed dimensions"))

neo4j_vector = Neo4jVectorStore(
    username=username, 
    password=password, 
    url=url, 
    embedding_dimension=embed_dim,
    database=database,
    index_name='vector'
)

In [20]:
from llama_index.storage.storage_context import StorageContext
from llama_index import VectorStoreIndex

storage_context_vector = StorageContext.from_defaults(vector_store=neo4j_vector)
vector_index = VectorStoreIndex.from_documents(
    documents, 
    storage_context=storage_context_vector, 
    service_context=service_context,
)

At this stage we can check with a cypher query that the stored embeddings have the right dimension 384
```cypher
MATCH (n:Chunk) RETURN size(n.embedding)
```
And make sure your Neo4j index "vector" has the right dimension in its config
```cypher
SHOW INDEXES
YIELD name, type, indexProvider AS provider, options, createStatement
RETURN name, type, provider, options.indexConfig AS config, createStatement
```

### Conclusions from building the Vector DB
- there are 16 chunks for 1 wikipedia page, they contain the text, the embedding, and ids
- there are no relationships
- the chunks are not related to entities at all
- the chunks get dumped into the same Graph database than the entity graph (unless you pass a different parameter), but for simplicity with Bloom viz, I chose to use the same database
- the embedding is the embedding for the chunk which can be more or less long so more or less "averaged"

## 4. (Optional) Persist and Load from disk Llama Indexes
### 4.1. Persist

### 4.2. Restore
In Llama Index, there are two scenarios we could apply Graph RAG:

- Build Knowledge Graph from documents with Llama Index, with LLM or even local models, to do this, we should go for `KnowledgeGraphIndex`.

- Leveraging existing Knowledge Graph, in this case, we should use `KnowledgeGraphRAGQueryEngine`.



In [31]:
existing_vector = Neo4jVectorStore(
    username,
    password,
    url,
    384,
    index_name="vector",
    text_node_property="text",
    embedding_node_property="embedding"
)
vector_index = VectorStoreIndex.from_vector_store(
    existing_vector,
    service_context=service_context,
)

## 5. Prepare for different query approaches

### 5.1 Graph RAG query engine
#### 5.1.1 Choose a retriever

There are issues with the llama_index docs as I describe [here](https://github.com/run-llama/llama_index/issues/10474)

reference:

- [KGTableRetriever](https://docs.llamaindex.ai/en/stable/api_reference/query/retrievers/kg.html#llama_index.indices.knowledge_graph.retrievers.KGTableRetriever) ??
- [KnowledgeGraphRAGRetriever](https://docs.llamaindex.ai/en/stable/api_reference/query/retrievers/kg.html#llama_index.indices.knowledge_graph.retrievers.KnowledgeGraphRAGRetriever) "Retriever that perform SubGraph RAG towards knowledge graph."
- [RetrieverQueryEngine](https://docs.llamaindex.ai/en/stable/api_reference/query/query_engines/retriever_query_engine.html)

See [Retriever Modes](https://docs.llamaindex.ai/en/stable/module_guides/querying/retriever/retriever_modes.html) for a full list of (index-specific) retriever modes and the retriever classes they map to.

For `KnowledgeGraphIndex`, we have a choice of `keyword`, `embedding` or `hybrid`


In [42]:
retriever = kg_index.as_retriever(
    retriever_mode="keyword" # ="embedding" # ="hybrid"
)
nodes = retriever.retrieve("Tell me about Peter Quill.")
for node in nodes:
    display(Markdown(node.text))

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: 5401f462-d91a-4e71-98b7-6f43efb5b6d0: === Filming ===
Principal photography began on November 8, 2021, at Trilith S...


=== Filming ===
Principal photography began on November 8, 2021, at Trilith Studios in Atlanta, Georgia, under the working title Hot Christmas. Henry Braham serves as cinematographer, after doing so for Vol. 2, The Suicide Squad,  and The Guardians of the Galaxy Holiday Special. Filming was previously scheduled to begin in January or February 2019 prior to Gunn's firing, and then in February 2021, before Gunn began work on Peacemaker. With the start of filming, Sylvester Stallone revealed that he would return as Stakar Ogord from Vol. 2, and Gunn posted a photo of the main cast members which revealed that Chukwudi Iwuji was part of the film following his collaboration with Gunn on Peacemaker. Iwuji's screen test for the film was shot on the set of Peacemaker with that series' crew, and Marvel repaid this favor by letting Gunn use the Vol. 3 set and crew to film Ezra Miller's cameo appearance as Barry Allen / The Flash for the Peacemaker season finale.Production designer Beth Mickle said Gunn chose to mainly use practical effects for Vol. 3 after they did so with their work on The Suicide Squad. In February 2021, Gunn stated the film would be shot using Industrial Light & Magic's StageCraft virtual production technology that was developed for the Disney+ Star Wars series The Mandalorian, but in October, he said they would not be able to use the technology because the sets were too big, believing they were larger than the sets used on The Suicide Squad. The interior of the Guardians' new ship, the Bowie, was a four-story set. Judianna Makovsky serves as costume designer. The Guardians of the Galaxy Holiday Special was filmed at the same time as Vol. 3, from February to late April 2022, with the same main cast and sets. Gunn enjoyed being able to switch to filming the special after doing scenes for Vol. 3, given the tonal difference between the two with Vol. 3 being more "emotional", feeling it helped provide a "relief" to the actors as well, and called the Holiday Special shoot easier than Vol. 3. In February 2022, Callie Brand was revealed to appear in the film as an alien. Shooting was also expected to occur in London, England, in late 2021. Vol. 3 is the first MCU film to feature the word "fuck" uncensored; it is spoken by Quill. The dialogue "Open the fucking door" was not scripted, but improvised by Pratt on Gunn's suggestion, though there was some consideration over giving the one-liner to Groot instead. Gunn mentioned that the Korean film The Villainess (2017) inspired one of the fight scenes in Vol. 3. Filming on Vol. 3 wrapped on May 6, 2022.

The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
['SPEAKS_UNCENSORED_"FUCK"_IN', 'Vol. 3']

In [45]:
from llama_index.retrievers import KnowledgeGraphRAGRetriever

graph_rag_retriever = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    service_context=service_context,
    llm=llm,
    verbose=True,
    # graph_query_synthesis_prompt=
    # graph_response_answer_prompt=
)
nodes = retriever.retrieve("Tell me about Peter Quill.")
for node in nodes:
    display(Markdown(node.text))

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: 5401f462-d91a-4e71-98b7-6f43efb5b6d0: === Filming ===
Principal photography began on November 8, 2021, at Trilith S...
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: 32627b8e-e016-474e-b5fe-808e6f5c47e5: Guardians of the Galaxy Vol. 3 (stylized in marketing as Guardians of the Gal...
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: 9522464d-0f5e-423a-a26f-b378af1c496a: == Cast ==
Chris Pratt as Peter Quill / Star-Lord:The half-human, half-Celest...


=== Filming ===
Principal photography began on November 8, 2021, at Trilith Studios in Atlanta, Georgia, under the working title Hot Christmas. Henry Braham serves as cinematographer, after doing so for Vol. 2, The Suicide Squad,  and The Guardians of the Galaxy Holiday Special. Filming was previously scheduled to begin in January or February 2019 prior to Gunn's firing, and then in February 2021, before Gunn began work on Peacemaker. With the start of filming, Sylvester Stallone revealed that he would return as Stakar Ogord from Vol. 2, and Gunn posted a photo of the main cast members which revealed that Chukwudi Iwuji was part of the film following his collaboration with Gunn on Peacemaker. Iwuji's screen test for the film was shot on the set of Peacemaker with that series' crew, and Marvel repaid this favor by letting Gunn use the Vol. 3 set and crew to film Ezra Miller's cameo appearance as Barry Allen / The Flash for the Peacemaker season finale.Production designer Beth Mickle said Gunn chose to mainly use practical effects for Vol. 3 after they did so with their work on The Suicide Squad. In February 2021, Gunn stated the film would be shot using Industrial Light & Magic's StageCraft virtual production technology that was developed for the Disney+ Star Wars series The Mandalorian, but in October, he said they would not be able to use the technology because the sets were too big, believing they were larger than the sets used on The Suicide Squad. The interior of the Guardians' new ship, the Bowie, was a four-story set. Judianna Makovsky serves as costume designer. The Guardians of the Galaxy Holiday Special was filmed at the same time as Vol. 3, from February to late April 2022, with the same main cast and sets. Gunn enjoyed being able to switch to filming the special after doing scenes for Vol. 3, given the tonal difference between the two with Vol. 3 being more "emotional", feeling it helped provide a "relief" to the actors as well, and called the Holiday Special shoot easier than Vol. 3. In February 2022, Callie Brand was revealed to appear in the film as an alien. Shooting was also expected to occur in London, England, in late 2021. Vol. 3 is the first MCU film to feature the word "fuck" uncensored; it is spoken by Quill. The dialogue "Open the fucking door" was not scripted, but improvised by Pratt on Gunn's suggestion, though there was some consideration over giving the one-liner to Groot instead. Gunn mentioned that the Korean film The Villainess (2017) inspired one of the fight scenes in Vol. 3. Filming on Vol. 3 wrapped on May 6, 2022.

Guardians of the Galaxy Vol. 3 (stylized in marketing as Guardians of the Galaxy Volume 3) is a 2023 American superhero film based on the Marvel Comics superhero team Guardians of the Galaxy, produced by Marvel Studios, and distributed by Walt Disney Studios Motion Pictures. It is the sequel to Guardians of the Galaxy (2014) and Guardians of the Galaxy Vol. 2 (2017), and the 32nd film in the Marvel Cinematic Universe (MCU). Written and directed by James Gunn, it features an ensemble cast including Chris Pratt, Zoe Saldaña, Dave Bautista, Karen Gillan, Pom Klementieff, Vin Diesel, Bradley Cooper, Will Poulter, Sean Gunn, Chukwudi Iwuji, Linda Cardellini, Nathan Fillion, and Sylvester Stallone. In the film, the Guardians must save Rocket's (Cooper) life from the High Evolutionary (Iwuji).
Gunn stated in November 2014 that he had initial ideas for a third and final film in the series, and announced his return to write and direct in April 2017. Disney fired him from the film in July 2018 following the resurfacing of controversial posts on Twitter, but the studio reversed course by that October and reinstated him. Gunn's return was publicly revealed in March 2019, with production resuming after he completed work for DC on The Suicide Squad (2021) and the first season of its spin-off series Peacemaker (2022). Filming began in November 2021 at Trilith Studios in Atlanta, Georgia, and lasted until May 2022.
Guardians of the Galaxy Vol. 3 premiered at Disneyland Paris on April 22, 2023, and was released in the United States on May 5, as part of Phase Five of the MCU. Like its predecessors, it was a critical and commercial success, with many deeming it to be a satisfactory conclusion to the trilogy. It grossed over $848.6 million worldwide, becoming the fourth-highest-grossing film of 2023. At the 96th Academy Awards, the film was nominated for Best Visual Effects.

== Cast ==
Chris Pratt as Peter Quill / Star-Lord:The half-human, half-Celestial leader of the Guardians of the Galaxy who was abducted from Earth as a child and raised by a group of alien thieves and smugglers, the Ravagers. In the film, Quill is in a "state of depression" following the appearance of a variant of his dead lover Gamora, who does not share the same affection for Quill as her older version had for him, which in turn affects his leadership of the Guardians.
Zoe Saldaña as Gamora:An orphan who seeks redemption for her past crimes, and was adopted and trained by Thanos to be his personal assassin. The original version of Gamora, a member of the Guardians, was killed by Thanos in Avengers: Infinity War (2018), and an alternate version of the character traveled to the present in Avengers: Endgame (2019); Saldaña reprises the latter role in this film, now serving as a member of the Ravagers. Saldaña stated that Vol. 3 would be the final time she would portray Gamora, noting that she originally signed to play her in one film and ended up playing the role for much longer, a role she was grateful to play due to the impact it especially had on female fans.
Dave Bautista as Drax the Destroyer:A member of the Guardians and a highly skilled warrior whose family was slaughtered by Ronan the Accuser, under the instructions of Thanos. Bautista stated that Vol. 3 would be the final time he would portray Drax, having been grateful for the role, while still calling it a "relief" to have concluded his time with the character, given the long hours needed to get into makeup and hoping to pursue more dramatic acting roles. Because of Bautista's decision, Gunn opted not to include Drax in the post-credits scene.
Karen Gillan as Nebula:A member of the Guardians, a former Avenger, and Gamora's adoptive sister who, similarly to her, was trained by their adoptive father Thanos to be his personal assassin. Gillan believed Nebula was developing into a "slightly different person" with more levity as she starts to heal psychologically following the death of Thanos, who was the source of her abuse and torment. Vol. 3 fulfills a character arc for the character writer and director James Gunn envisioned when starting work on Guardians of the Galaxy (2014), going from a minor villain to a member of the Guardians. Although the film teases a possible romance between Star-Lord and Nebula, Gunn denies having ever considered the two becoming a couple, though Gillan does believe she harbors a small crush on Quill.
Pom Klementieff as Mantis: A member of the Guardians with empathic powers, and Quill's half-sister.
Vin Diesel as Groot: A member of the Guardians who is a tree-like humanoid and the accomplice of Rocket. Austin Freeman provided the motion-capture for Groot.
Bradley Cooper as Rocket:A member of the Guardians and a former Avenger who is a genetically engineered raccoon-based bounty hunter and a master of weapons and military tactics. Gunn said that the film tells Rocket's story, including his background and "where he's going", along with how that ties into the other Guardians and the end of this iteration of the team. The film completes a character arc that was established in Guardians of the Galaxy and Guardians of the Galaxy Vol. 2 (2017), and continued in Infinity War and Endgame. Sean Gunn once again provided on set motion capture for the character, while also voicing young Rocket. Cooper also voiced adolescent Rocket while Noah Raskin voiced baby Rocket.
Will Poulter as Adam Warlock:A powerful artificial being created by the Sovereign to destroy the Guardians. Given Warlock is newly born from the Sovereign's cocoon, he is "basically a baby" that "does not understand life very well". Poulter believed there was "a lot of comedy" in someone just entering the world for the first time and "trying to develop his moral compass", while also having "some genuine pathos". Gunn thought Warlock's interactions with the Guardians provided "an interesting juxtaposition" to what their journey has been, and described him as a more traditional superhero compared to the Guardians, although not necessarily a hero.
Sean Gunn as Kraglin: A member of the Guardians and Yondu Udonta's former second-in-command in the Ravagers.

The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
('Chris pratt', 'Plays character of', 'Peter quill / star-lord')
('Quill', 'Speaks uncensored "fuck" in', 'Vol. 3')
['SPEAKS_UNCENSORED_"FUCK"_IN', 'Vol. 3']

#### 5.1.2 Build a query engine

In [27]:
# in case we just built the index and we have it available
kg_rag_query_engine = kg_index.as_query_engine(
    include_text=False, 
    retriever_mode="hybrid",
    response_mode="tree_summarize"
)

# from llama_index.query_engine import RetrieverQueryEngine

# kg_rag_query_engine = RetrieverQueryEngine.from_args(
#     graph_rag_retriever,
#     service_context=service_context,
# )

In [28]:
from llama_index.indices.knowledge_graph.retrievers import KGTableRetriever

assert type(kg_rag_query_engine.retriever) == KGTableRetriever

### 5.2. Vector RAG query engine

In [21]:
vector_rag_query_engine = vector_index.as_query_engine()

In [22]:
from llama_index.query_engine.retriever_query_engine import RetrieverQueryEngine

assert type(vector_rag_query_engine) == RetrieverQueryEngine

### 5.3 Graph+Vector RAG query engine

In [30]:
# import QueryBundle
from llama_index import QueryBundle

# import NodeWithScore
from llama_index.schema import NodeWithScore

# Retrievers
from llama_index.retrievers import BaseRetriever, VectorIndexRetriever, KGTableRetriever

from typing import List


class CustomRetriever(BaseRetriever):
    """Custom retriever that performs both Vector search and Knowledge Graph search"""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        kg_retriever: KGTableRetriever,
        mode: str = "OR",
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever
        self._kg_retriever = kg_retriever
        if mode not in ("AND", "OR"):
            raise ValueError("Invalid mode.")
        self._mode = mode

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        kg_nodes = self._kg_retriever.retrieve(query_bundle)

        vector_ids = {n.node.node_id for n in vector_nodes}
        kg_ids = {n.node.node_id for n in kg_nodes}

        combined_dict = {n.node.node_id: n for n in vector_nodes}
        combined_dict.update({n.node.node_id: n for n in kg_nodes})

        if self._mode == "AND":
            retrieve_ids = vector_ids.intersection(kg_ids)
        else:
            retrieve_ids = vector_ids.union(kg_ids)

        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes

In [34]:
from llama_index import get_response_synthesizer
from llama_index.query_engine import RetrieverQueryEngine

# create custom retriever
vector_retriever = VectorIndexRetriever(index=vector_index)
kg_retriever = KGTableRetriever(
    index=kg_index, retriever_mode="keyword", include_text=False
)
custom_retriever = CustomRetriever(vector_retriever, kg_retriever)

# create response synthesizer
response_synthesizer = get_response_synthesizer(
    service_context=service_context,
    response_mode="tree_summarize",
)

In [35]:
graph_vector_rag_query_engine = RetrieverQueryEngine(
    retriever=custom_retriever,
    response_synthesizer=response_synthesizer,
)

## 6. Query with the engines

In [24]:
from IPython.display import Markdown, display

In [29]:
response_graph_rag = kg_rag_query_engine.query("Tell me about Peter Quill.")

display(Markdown(f"<b>{response_graph_rag}</b>"))

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


<b> Peter Quill is a subject in the provided knowledge sequence, specifically connected to the object "Vol. 3" through the relationship "SPEAKS\_UNCENSORED\_'FUCK'\_IN". However, without further context or information, it is unclear what specific properties or characteristics belong to Peter Quill based on the given schema. Therefore, I cannot provide a comprehensive answer about Peter Quill beyond this connection.</b>

In [25]:
response_vector_rag = vector_rag_query_engine.query("Tell me about Peter Quill.")

display(Markdown(f"<b>{response_vector_rag}</b>"))

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


<b> Peter Quill is a character in the Marvel Cinematic Universe, portrayed by Chris Pratt. He is the half-human, half-Celestial leader of the Guardians of the Galaxy. Quill was abducted from Earth as a child and raised by a group of alien thieves and smugglers, the Ravagers. In the films, Quill experiences depression following the appearance of an alternate version of his deceased lover Gamora, who does not share the same affections towards him. This affects his leadership of the Guardians.</b>

In [36]:
response_graph_vector_rag = graph_vector_rag_query_engine.query("Tell me about Peter Quill.")

display(Markdown(f"<b>{response_graph_vector_rag}</b>"))

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


<b> Peter Quill, also known as Star-Lord, is a character in the Marvel Cinematic Universe who serves as the leader of the Guardians of the Galaxy. He was abducted from Earth as a child and raised by a group of alien thieves and smugglers, the Ravagers. In the films, Quill is depicted as being in a state of depression following the appearance of an alternate version of his dead lover Gamora, who does not reciprocate his affections. He was portrayed by Chris Pratt in the first two Guardians of the Galaxy films and Avengers: Endgame. Pratt stated that he would be wrapping up his time with the character due to his contract ending and wanting to explore other roles.</b>

## 7. Comparison and Conclusion

In [37]:
analysis = llm.complete(f"""
Compare the QA results on "Tell me about Peter Quill.", list the knowledge facts between them, to help evalute them. Output in markdown table.

Result from Graph: {response_graph_rag}
---
Result from Vector: {response_vector_rag}
---
Result Graph+Vector: {response_graph_vector_rag}
---

""")

INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"


In [38]:
display(Markdown(analysis.text))

 | Fact | Result from Graph | Result from Vector | Result Graph+Vector |
|-------|--------------------|---------------------|----------------------|
| Name  | Peter Quill         | Peter Quill          | Peter Quill, Star-Lord |
| Role  | Leader of the Guardians of the Galaxy | Character in Marvel Cinematic Universe | Leader of the Guardians of the Galaxy |
| Connection to Object | Speaks uncensored 'FUCK' in Vol. 3 | Raised by Ravagers, abducted as a child | Abducted from Earth as a child, raised by the Ravagers |
| Affiliation | - | Leader of Guardians of the Galaxy, Celestial-human hybrid | Leader of Guardians of the Galaxy |
| Depiction in films | Unclear | Portrayed by Chris Pratt, experiences depression over Gamora | Portrayed by Chris Pratt, experiences depression over alternate Gamora |
| Background | Half-human, half-Celestial, connection to Vol. 3 through 'Speaks\_UNCENSORED\_'FUCK'\_IN' | Originated on Earth, abducted as a child, raised by Ravagers | Originated on Earth, abducted as a child, raised by Ravagers |
| Emotional State | - | Experiences depression following appearance of alternate Gamora | Experiences depression following appearance of alternate Gamora |
| Contractual Status | - | - | Pratt's contract ending, wanting to explore other roles |